<a href="https://colab.research.google.com/github/ozturkgizem/gtsdb/blob/main/gtsdb_yolov4_son.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Verisetini Kaggle'dan Çekmek İçin Gerekli İşlemler


In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download hieunguyen21/german-traffic-sign-detection-benchmark-gtsdb

In [ ]:
! unzip german-traffic-sign-detection-benchmark-gtsdb.zip

## YOLO FORMATTING

In [ ]:
import pandas as pd
import os
import cv2

CLASSES NAMES

In [ ]:
classes = {0:'prohibitory',
           1:'danger',
           2:'mandatory',
           3:'other'}
           
c = pd.DataFrame([classes])
c.to_csv('/content/classes.names', header=False, index=False, sep='\n')

ANNOTATIONS

In [ ]:
# Prohibitory category: circular Traffic Signs with white background and red border line
p = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]

# Danger category: triangular Traffic Signs with white background and red border line
d = [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

# Mandatory category: circular Traffic Signs with blue background
m = [33, 34, 35, 36, 37, 38, 39, 40]

# Other category:
o = [6, 12, 13, 14, 17, 32, 41, 42]

In [ ]:
# Her fotoğraf için annotation txt dosyalarının oluşturulması için ön hazırlık
ann = pd.read_csv('/content/FullIJCNN2013/gt.txt',
                  names=['ImageID',
                         'XMin',
                         'YMin',
                         'XMax',
                         'YMax',
                         'ClassID'],
                  sep=';')

# Adding new empty columns to dataFrame to save numbers for YOLO format
ann['CategoryID'] = ''
ann['center x'] = ''
ann['center y'] = ''
ann['width'] = ''
ann['height'] = ''

# Getting category's ID according to the class's ID
ann.loc[ann['ClassID'].isin(p), 'CategoryID'] = 0
ann.loc[ann['ClassID'].isin(d), 'CategoryID'] = 1
ann.loc[ann['ClassID'].isin(m), 'CategoryID'] = 2
ann.loc[ann['ClassID'].isin(o), 'CategoryID'] = 3

# Calculating bounding box's center in x and y for all rows
# Saving results to appropriate columns
ann['center x'] = (ann['XMax'] + ann['XMin']) / 2
ann['center y'] = (ann['YMax'] + ann['YMin']) / 2

# Calculating bounding box's width and height for all rows
# Saving results to appropriate columns
ann['width'] = ann['XMax'] - ann['XMin']
ann['height'] = ann['YMax'] - ann['YMin']

# Getting Pandas dataFrame that has only needed columns
# By using 'loc' method we locate here all rows but only specified columns
# By using copy() we create separate dataFrame not just a reference to the previous one and, in this way, initial dataFrame will not be changed
r = ann.loc[:, ['ImageID',
                'CategoryID',
                'center x',
                'center y',
                'width',
                'height']].copy()

# Check point
# Showing first 5 rows from the dataFrame
print(r.head())

     ImageID CategoryID  center x  center y  width  height
0  00000.ppm          1     794.5     428.5     41      35
1  00001.ppm          2    1003.5     410.0     41      44
2  00001.ppm          2     414.0     523.0     56      58
3  00001.ppm          3    1002.0     362.5     58      55
4  00002.ppm          2     949.0     534.0    114     116


In [ ]:
'''Normalizing YOLO numbers according to the real image width and height
Saving annotations in txt files
Converting images from ppm to jpg'''

def annotation(fullPathtoDataset):
  os.chdir(fullPathtoDataset)

  for current_dir, dirs, files in os.walk('.'):
     for f in files:
        if f.endswith('.ppm'):
          image_ppm = cv2.imread(f)
          h, w = image_ppm.shape[:2]

          # Slicing only name of the file without extension
          image_name = f[:-4]

          sub_r = r.loc[r['ImageID'] == f].copy()

          # Normalizing calculated bounding boxes' coordinates according to the real image width and height
          sub_r['center x'] = sub_r['center x'] / w
          sub_r['center y'] = sub_r['center y'] / h
          sub_r['width'] = sub_r['width'] / w
          sub_r['height'] = sub_r['height'] / h

          resulted_frame = sub_r.loc[:, ['CategoryID',
                                         'center x',
                                         'center y',
                                         'width',
                                         'height']].copy()

          # Checking if there is no any annotations for current image
          if resulted_frame.isnull().values.all():
              continue

          # Preparing path where to save txt file
          path_to_save = fullPathtoDataset + '/' + image_name + '.txt'

          # Saving resulted Pandas dataFrame into txt file
          resulted_frame.to_csv(path_to_save, header=False, index=False, sep=' ')

          # Preparing path where to save jpg image
          path_to_save = fullPathtoDataset + '/' + image_name + '.jpg'

          # Saving image in jpg format by OpenCV function that uses extension to choose format to save with
          cv2.imwrite(path_to_save, image_ppm)

TXT VE JPGLERİN OLUŞTURULMASI

In [ ]:
annotation('/content/FullIJCNN2013')

In [ ]:
#from pathlib import Path

#rootdir = '/content/FullIJCNN2013'
#for path in Path(rootdir).iterdir():
#  if path.is_dir():
#    path=str(path)
#    annotation(path)

TRAİN VE TEST TXTLERİN OLUŞTURULMASI

In [ ]:
full_path_to_images = '/content/FullIJCNN2013'

os.chdir(full_path_to_images)

# Defining list to write paths in
p = []

for current_dir, dirs, files in os.walk('.'):
    # Going through all files
    for f in files:
        # Checking if filename ends with '.jpg'
        if f.endswith('.jpg'):
            path_to_save_into_txt_files = full_path_to_images + '/' + f
            p.append(path_to_save_into_txt_files + '\n')


# Slicing first 15% of elements from the list to write into the test.txt file
p_test = p[:int(len(p) * 0.15)]

# Deleting from initial list first 15% of elements
p = p[int(len(p) * 0.15):]

with open('train.txt', 'w') as train_txt:
    # Going through all elements of the list
    for e in p:
        train_txt.write(e)

with open('test.txt', 'w') as test_txt:
    # Going through all elements of the list
    for e in p_test:
        test_txt.write(e)

# Darknet


In [ ]:
os.chdir('/content')
! git clone https://github.com/AlexeyAB/darknet/

Cloning into 'darknet'...
remote: Enumerating objects: 15386, done.
remote: Total 15386 (delta 0), reused 0 (delta 0), pack-reused 15386
Receiving objects: 100% (15386/15386), 14.01 MiB | 16.53 MiB/s, done.
Resolving deltas: 100% (10345/10345), done.


In [ ]:
os.chdir('/content/darknet')
#compiling w cuda
! sed -i 's/GPU=0/GPU=1/g' Makefile

! sed -i 's/CUDNN=0/CUDNN=1/g' Makefile

#compiling w openCv
import cv2
! sed -i 's/OPENCV=0/OPENCV=1/g' Makefile

In [ ]:
# CUSTOM_DATA.DATA FILE

with open('/content/darknet/cfg/ts_data.data', 'w') as out:
  out.write('classes = 4\n')
  out.write('train = /content/FullIJCNN2013/train.txt\n')
  out.write('valid = /content/FullIJCNN2013/test.txt\n')
  out.write('names = /content/classes.names\n')
  out.write('backup = backup')

In [ ]:
os.chdir('/content/darknet')
! make

In [ ]:
!./darknet

usage: ./darknet <function>


In [ ]:
#!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

In [ ]:
#!./darknet detector train cfg/ts_data.data cfg/yolov4-custom_train.cfg yolov4.conv.137 -thresh 0.25 -dont_show 
#!./darknet detector train cfg/ts_data.data cfg/yolov4-tiny_train.cfg yolov4-tiny.weights -thresh 0.25 -dont_show

In [ ]:
#train devam
!./darknet detector train cfg/ts_data.data cfg/yolov4-tiny_train.cfg yolov4-tiny_train_6000.weights -thresh 0.25 -dont_show

In [ ]:
!./darknet detector map cfg/ts_data.data cfg/yolov4-tiny_train.cfg backup/yolov4-tiny_train_8000.weights

foto

In [ ]:
!./darknet detector test cfg/ts_data.data cfg/yolov4-tiny_train.cfg yolov4-tiny_train_8000.weights content/image1.jpg -thresh 0.3
imShow('predictions.jpg')

video

In [ ]:
!./darknet detector demo cfg/ts_data.data cfg/yolov4-tiny_train.cfg yolov4-tiny_train_8000.weights -dont_show /mydrive/mask_test_videos/test1.mp4 -thresh 0.7 -i 0 -out_filename /mydrive/mask_test_videos/results1.avi